<h1>Modelisation</h1>

<h3>Plan: </h3>
<ol>
    <li>Pré-traitement</li>
    <ul>
        <li>Transformation des variables qualitatives en dummies</li>
        <li>Transformation Logarithme</li>
    </ul>
    <li>Fractionnement des données</li>
    <ul>
        <li>Fractionnement</li>
        <li>Definition des données train et test</li>
        <li>Fonction utiles</li>
    </ul>
    <li>Modelisation</li>
    <ul>
        <li>Regression Linéaire</li>
        <li>Regression Ridge</li>
        <li>Decision Tree</li>
        <li>Random Forest</li>
        <li>Gradient Boosting</li>
        <li>Xgboost</li>
        <li>Staking</li>
    </ul>
    <li>Conclusion</li>
</ol>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV,RandomizedSearchCV

In [2]:
df = pd.read_csv('../data/data_v3.csv',sep=';')

In [3]:
df.head()

,Bathrooms,Bedrooms,Erf_size,Floor_size,Garages,Listing_date,Listing_number,Pet_friendly,Price,Town,...,Flatlet,Fibre_internet,Furnished_optional,ADSL_internet,Satellite_internet,Fixed_wimax_internet,month,year,District,Price_bins
0,3.0,3.0,304.0,326.0,2.0,2020-07-10,108859219,1.0,6480000.0,"Tokai, Cape Town",...,0.0,0.0,0.0,0.0,0.0,0.0,7,2020,Tokai,"(0.0, 10000000.0]"
1,3.0,3.0,430.5,140.0,2.0,2020-09-23,109153971,1.0,6250000.0,"14 Graham Road, Sea Point, Cape Town",...,0.0,0.0,0.0,0.0,0.0,0.0,9,2020,Sea Point,"(0.0, 10000000.0]"
2,3.5,4.0,659.0,300.0,2.0,2020-09-23,109154367,0.0,13950000.0,"Camps Bay, Cape Town",...,0.0,0.0,0.0,0.0,0.0,0.0,9,2020,Camps Bay,"(10000000.0, 20000000.0]"
3,1.0,2.0,430.5,92.0,1.0,2020-09-23,109154455,0.0,1050000.0,"Ottery, Cape Town",...,0.0,0.0,0.0,0.0,0.0,0.0,9,2020,Ottery,"(0.0, 10000000.0]"
4,1.0,1.0,430.5,61.0,1.0,2020-07-23,108907258,0.0,1265000.0,"Muizenberg, Cape Town",...,0.0,0.0,0.0,0.0,0.0,0.0,7,2020,Muizenberg,"(0.0, 10000000.0]"


<h2>1) Pré-traitement</h2>

<ul><li><h3>Transformation des varibles qualitatives en dummies</h3></li></ul>

In [4]:
var_qualitative = ["District","Type_of_property"]
dummies = pd.get_dummies(df[var_qualitative])
df=pd.concat([df,dummies],axis=1)

In [5]:
df.head()

,Bathrooms,Bedrooms,Erf_size,Floor_size,Garages,Listing_date,Listing_number,Pet_friendly,Price,Town,...,District_Waterfront,District_Woodstock,District_Wynberg,District_Wynberg Upper,District_Zonnebloem,District_Zwaanswyk,District_others,Type_of_property_Apartment / Flat,Type_of_property_House,Type_of_property_Townhouse
0,3.0,3.0,304.0,326.0,2.0,2020-07-10,108859219,1.0,6480000.0,"Tokai, Cape Town",...,0,0,0,0,0,0,0,0,1,0
1,3.0,3.0,430.5,140.0,2.0,2020-09-23,109153971,1.0,6250000.0,"14 Graham Road, Sea Point, Cape Town",...,0,0,0,0,0,0,0,0,0,1
2,3.5,4.0,659.0,300.0,2.0,2020-09-23,109154367,0.0,13950000.0,"Camps Bay, Cape Town",...,0,0,0,0,0,0,0,0,1,0
3,1.0,2.0,430.5,92.0,1.0,2020-09-23,109154455,0.0,1050000.0,"Ottery, Cape Town",...,0,0,0,0,0,0,0,1,0,0
4,1.0,1.0,430.5,61.0,1.0,2020-07-23,108907258,0.0,1265000.0,"Muizenberg, Cape Town",...,0,0,0,0,0,0,0,1,0,0


<ul><li><h3>Transformation Logarithme</h3></li></ul>
<p>Cette  transformation est utile pour que la distribution soit proche de loi normale</p>

In [6]:
df["Bathrooms_log"] = np.log(df['Bathrooms']+1)
df["Bedrooms_log"]  = np.log(df['Bedrooms']+1)
df["Floor_size_log"]= np.log(df['Floor_size']+1)
df["Erf_size_log"]  = np.log(df['Erf_size']+1)
df["Garages_log"]   = np.log(df['Garages']+1)
df["Secure_parking_log"] = np.log(df['Secure_parking']+1)
df["Parking_log"]   = np.log(df['Parking']+1)
df["Price_log"]     = np.log(df['Price'])

<h2>2 ) Fractionnement des données</h2>

<ul><li><h3>Fractionnement</h3></li></ul>

In [7]:
##train:80% et test:20%
np.random.seed(10)
train,test=train_test_split(df,test_size=0.2,random_state=0)

<p>Pour la regression linéaire et Ridge, les données doivent suivre la loi Normal, c'est a dire qu'il faut utiliser les variable issue du Log Transform pour plus de performance</p>
<p>Pour les autres models, on n'utilisera pas les variables avec log</p>

<ul><li><h3>Definition des données train et test</h3></li></ul>

In [8]:
#Variable non significatif, qualitatif(non dummies) ou Label
not_used_features = [
            'Listing_date',
            'Listing_number', 
            'Price',
            'Town', 
            'month',
            'year', 
            'District',
            'Price_bins',
            'Type_of_property', 
            'Price_log',
            'Price',
        ]

simple_features = [
                    'Bathrooms',
                    'Bedrooms',
                    'Floor_size',
                    'Erf_size',
                    'Garages',
                    'Secure_parking',
                    'Parking',
                ]
log_features =[
                    'Bathrooms_log',
                    'Bedrooms_log',
                    'Floor_size_log',
                    'Erf_size_log',
                    'Garages_log',
                    'Secure_parking_log',
                    'Parking_log',
                    
                ]
                
features_with_log       = [x for x in df.columns.tolist() if x not in simple_features and x not in not_used_features]
features_without_log    = [x for x in df.columns.tolist() if x not in log_features and x not in not_used_features]

In [9]:
### Donnée pour La regression linéaire et ridge
X_train_log   = train[features_with_log]
y_train_log   = train['Price_log']
X_test_log    = test[features_with_log]
y_test_log    = test['Price_log']

In [10]:
### Donnée pour les autres models
X_train = train[features_without_log]
y_train = train['Price_log']
X_test = test[features_without_log]
y_test = test['Price_log']

<h2>3) Modelisation</h2>

<ul><li><h3>Regression Linéaire</h3></li></ul>

In [13]:
regr = linear_model.LinearRegression()
regr.fit(X_train_log, y_train_log)

y_pred = regr.predict(X_test_log)
train_pred = regr.predict(X_train_log)

trainrmse = np.sqrt(metrics.mean_squared_error(y_train_log, train_pred ))
rmse = np.sqrt(metrics.mean_squared_error(y_test_log, y_pred))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

RMSE train : 0.33212611235040906
RMSE test : 3213819722.9759736


<ul><li><h3>Regression Ridge</h3></li></ul>

In [15]:
model = Ridge()
grid = dict()
grid['alpha'] = np.arange(0.02, 1, 0.01)

search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
ridge = search
ridge.fit(X_train_log, y_train_log)
y_pred_ridge    = ridge.predict(X_test_log)
train_pred      = ridge.predict(X_train_log)
trainrmse       = np.sqrt(metrics.mean_squared_error(y_train_log, train_pred ))
rmse            = np.sqrt(metrics.mean_squared_error(y_test_log, y_pred_ridge))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

RMSE train : 0.33212734548175565
RMSE test : 0.336867115156158


In [16]:
ridge.best_params_

{'alpha': 0.06}

<ul><li><h3>Decision tree</h3></li></ul>

In [19]:
dt = DecisionTreeRegressor(random_state=0)
dt.fit(X_train,y_train)
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)
train_rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred ))
test_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print("RMSE train :",train_rmse)
print("RMSE test :",test_rmse)

RMSE train : 0.044511809432516744
RMSE test : 0.43184374877380255


<ul><li><h3>Random forest</h3></li></ul>

In [20]:

rf = RandomForestRegressor()
rf.fit(X_train,y_train)
y_train_pred    = dt.predict(X_train)
y_test_pred     = dt.predict(X_test)
train_rmse      = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred ))
test_rmse       = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print("RMSE train :",train_rmse)
print("RMSE test :",test_rmse)

RMSE train : 0.044511809432516744
RMSE test : 0.43184374877380255


In [21]:
### Random forest avec tunning Random Search

parameters = {
                'bootstrap': [True],
                'max_depth': [None, 10, 20, 30],
                'max_features': range(20, 27),
                'min_samples_leaf': [3, 4, 5,1],
                'min_samples_split': [2,8, 10, 12],
                'n_estimators': [100, 200, 300, 1000]
            }
search  = RandomizedSearchCV(random_state=42,estimator=rf, param_distributions = parameters, cv = 5, n_jobs=-1).fit(X_train, y_train)
rf      = search.best_estimator_
print(rf)

test_pred   = rf.predict(X_test)
train_pred  = rf.predict(X_train)
trainrmse   = np.sqrt(metrics.mean_squared_error(y_train, train_pred ))
rmse        = np.sqrt(metrics.mean_squared_error(y_test, test_pred))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

RandomForestRegressor(max_depth=30, max_features=21, n_estimators=1000)
RMSE train : 0.15536059327667373
RMSE test : 0.31832117699199025


In [22]:
### Random forest Cross validation

kf      = KFold(10, shuffle=True, random_state=42).get_n_splits(X_train)
rmse    = np.sqrt(-cross_val_score(rf, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
print("std =",np.std(rmse))
print("mean=",np.mean(rmse))
print(rmse)

std = 0.014519582762890672
mean= 0.33402334457370625


<ul><li><h3>Gradient Boosting</h3></li></ul>

In [24]:
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)
y_train_pred    = gbr.predict(X_train)
y_test_pred     = gbr.predict(X_test)
train_rmse      = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred ))
test_rmse       = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print("RMSE train :",train_rmse)
print("RMSE test :",test_rmse)

RMSE train : 0.374229118528115
RMSE test : 0.3819963876056813


In [25]:
### Gradient Boosting with tunning

parameters = {  
                'learning_rate': [0.01,0.02,0.03],
                'subsample'    : [0.9, 0.5, 0.2, 0.1,1],
                'n_estimators' : [100,500,1000],
                'max_depth'    : [4,6,8,3]
            }
search = RandomizedSearchCV(random_state=42, estimator=gbr, param_distributions = parameters, cv = 5, n_jobs=-1).fit(X_train, y_train)
gbr = search.best_estimator_
print(gbr)

test_pred   = gbr.predict(X_test)
train_pred  = gbr.predict(X_train)
trainrmse   = np.sqrt(metrics.mean_squared_error(y_train, train_pred ))
rmse        = np.sqrt(metrics.mean_squared_error(y_test, test_pred))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

GradientBoostingRegressor(learning_rate=0.03, max_depth=6, n_estimators=1000,
                          subsample=0.9)
RMSE train : 0.1856319087239159
RMSE test : 0.2824625976256793


In [26]:
### Gradient Boosting cross validation

kf      = KFold(10, shuffle=True, random_state=42).get_n_splits(X_train)
rmse    = np.sqrt(-cross_val_score(gbr, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
print("std =",np.std(rmse))
print("mean=",np.mean(rmse))
print(rmse)

std = 0.011166680552356562
mean= 0.2854200397040854
[0.2998157  0.28567211 0.2868389  0.29519789 0.28420006 0.2928792
 0.28088358 0.25622654 0.28392945 0.28855698]


<ul><li><h3>Xgboost</h3></li></ul>

In [27]:
xgb = XGBRegressor()
xgb.fit(X_train,y_train)
train_pred = xgb.predict(X_train)
test_pred= xgb.predict(X_test)
train_rmse_xgb = np.sqrt(metrics.mean_squared_error(y_train, train_pred ))
rmse_xgb = np.sqrt(metrics.mean_squared_error(y_test, test_pred))
print("RMSE train :",train_rmse_xgb)
print("RMSE test :",rmse_xgb)

RMSE train : 0.20913142824125416
RMSE test : 0.2866707585744532


In [28]:
### Xgboost with tunning 

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5,0],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
search = RandomizedSearchCV(random_state=42, estimator=xgb, param_distributions = parameters, cv = 5, n_jobs=-1).fit(X_train, y_train)
xgb = search.best_estimator_

test_pred       = xgb.predict(X_test)
train_pred      = xgb.predict(X_train)
trainrmse       = np.sqrt(metrics.mean_squared_error(y_train, train_pred ))
rmse            = np.sqrt(metrics.mean_squared_error(y_test, test_pred))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

RMSE train : 0.2040485167529256
RMSE test : 0.2778018069092156


In [29]:
### Xgboost cross validation

kf      = KFold(10, shuffle=True, random_state=42).get_n_splits(X_train)
rmse    = np.sqrt(-cross_val_score(gbr, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
print("std =",np.std(rmse))
print("mean=",np.mean(rmse))
print(rmse)

std = 0.011577374474896521
mean= 0.28538169730726615
[0.30467038 0.28536548 0.28501065 0.29415153 0.28222266 0.2911139
 0.28032553 0.25699719 0.28452378 0.28943588]


<ul><li><h3>Stacking</h3></li></ul>

In [30]:
stacking = StackingRegressor([
                                ('XGB',xgb),
                                ('random_forest', rf),
                                ('GB', gbr)
                                
                             ],
                         final_estimator=LinearRegression(), cv=5)

In [31]:
stacking.fit(X_train, y_train)

StackingRegressor(cv=5,
                  estimators=[('XGB',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rate=0.03,
                                            max_delta_step=0, max_depth=6,
                                            min_child_weight=1, missing=nan,
                                            monotone_constraints='()',
                                            n_estimators=1000, n_jobs=8,
                                            num_p...dom_state=0,
                                            reg_alpha=0, reg_lambda=1,
           

In [32]:
test_pred   = stacking.predict(X_test)
train_pred  = stacking.predict(X_train)
trainrmse   = np.sqrt(metrics.mean_squared_error(y_train, train_pred ))
rmse        = np.sqrt(metrics.mean_squared_error(y_test, test_pred))
print("RMSE train :",trainrmse)
print("RMSE test :",rmse)

RMSE train : 0.19725153447599014
RMSE test : 0.27630052494312346


In [33]:
##Stacking cross validation

kf      = KFold(10, shuffle=True, random_state=42).get_n_splits(X_train)
rmse    = np.sqrt(-cross_val_score(stacking, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
print("std =",np.std(rmse))
print("mean=",np.mean(rmse))
print(rmse)

std = 0.010040133819838138
mean= 0.27946550476637366
[0.29583736 0.28280995 0.27711952 0.28143834 0.27920085 0.28332335
 0.27985123 0.25303549 0.28170755 0.28033141]


<h2>4) Conclusion</h2>

<p>Le stacking est la méthode de modélisation la plus efficace et la plus stable selon les évaluations réalisées avec :</p>
<ul>
    <li>RMSE train : 0.197</li>
    <li>RMSE test : 0.276</li>
    <li>std = 0.010  cross validation (ce qui montre bien la stabilité du modèle)</li>
</ul>
<p>malgré son coût élevé en temps d'entrainnement.</p>
<br/>

<p>Il est aussi possible d'améliorer les résultats en utilisant la "feature selection".</p>